In [1]:
using Pkg

Pkg.activate("..")
using Revise

using PeriDyn
using PDMesh

  Activating environment at `/mnt/D/git_repos/PeriDyn/Project.toml`
┌ Info: Precompiling PeriDyn [c3db0ce0-6d7e-41b5-be11-bcb6aafada44]
└ @ Base loading.jl:1342
  ** incremental compilation may be fatally broken for this module **



In [2]:
write_data("file.txt", delete(Cylinder(5, 2, 5), (a)-> a[:x][1, :] .> 0))

In [12]:
resolution = 1
out1 = create(Cuboid([0 50; 0 5; 0 5]), resolution=resolution, rand_=0.02, type=1)

hor = 3.0*resolution

mat_gen1 = GeneralMaterial(unpack(out1)..., hor, max_neigh=200)

K, G = 1.0, 1.0
den = 10000.0
cstretch = 0.5
mat_spec1 = OrdinaryStateBasedSpecific([K], [G], [cstretch], [den])
mat_spec2 = OrdinaryStateBasedSpecific([K], [G], [cstretch], [den])

block1 = PeridynamicsMaterial(mat_gen1, mat_spec1; name="block 1")

#

Average family members: 63.1408


OrdinaryStateBasedMaterial("block 1", 1:1, GeneralMaterial
type: [1]
horizon: 3.0
particle size: [1.0]
, OrdinaryStateBasedSpecific
bulk_modulus: [1.0]
shear_modulus: [1.0]
critical_stretch: [0.5]
density: [10000.0]
)

In [13]:
k = 100000.0
blocks = [block1]

# mask1 = (block1.general.x[1,:] .< 1 ) .& (block1.general.x[2,:] .< 0)
# mask2 = (block1.general.x[1,:] .< 1 ) .& (block1.general.x[2,:] .> 0)
mask2 = block1.general.x[1,:] .< 5 

bc = [
    # FixBC(mask1),
    ToFroBC(mask2, [0.0, -0.001, 0.0], 1000),
    ]

RM = [
        [LinearRepulsionModel(k, b1, distanceX=3, max_neighs=200) for b1 in blocks]..., 
        [LinearRepulsionModel(k, blocks[i], blocks[j], distanceX=3, max_neighs=200) for i in 2:length(blocks) for j in 1:i-1]...
    ]

env = Env(1, blocks, RM, bc, 0.5)

nothing
#

In [ ]:
velocity_verlet!([env], 50000, filewrite_freq=10, neigh_update_freq=10, file_prefix="half-cylinder", start_at=0)


Updating neighbors for collision..................
Average repulsive neighs: 0.0
Done
Material force calculation passed (block 1). [Undeformed]
Time step: 0
Momentum: 1[0.0; 0.0; 0.0]

Writting data file................................
Done
0.002%
0.004%
0.006%
0.008%
0.01%
0.012%
0.014%
0.016%
0.018%
Momentum: 10[-0.0003738458896473038; -1250.1129617196307; -0.00015622098515352505]

Writting data file................................
Done

Updating neighbors for collision..................
Average repulsive neighs: 0.0032
0.02%
0.022%
0.024%
0.026%
0.028%
0.03%
0.032%
0.034%
0.036%
0.038%
Momentum: 20[-0.0031371864237347688; -1250.6895463103199; -0.000949553431503661]

Writting data file................................
Done

Updating neighbors for collision..................
Average repulsive neighs: 0.0064
0.04%
0.042%
0.044%
0.046%
0.048%
0.05%
0.052%
0.054%
0.056%
0.058%
Momentum: 30[-0.010148001292406538; -1251.7455845642892; -0.002391444357169147]

Writting data file.............